# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key


In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../Resources/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.76,71,100,7.76,SH,1724616370
1,1,diffa,13.3154,12.6113,26.18,74,100,5.47,NE,1724616372
2,2,toliara,-23.3500,43.6667,23.46,62,0,2.54,MG,1724616373
3,3,amasra,41.7463,32.3863,22.11,78,2,2.28,TR,1724616374
4,4,saldanha,-33.0117,17.9442,10.03,88,10,5.08,ZA,1724616031


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,
    tiles = "OSM",
    size='Humidity',  # Size of the points based on humidity
    color='City',     # Optionally color by city for better distinction
    hover_cols=['city', 'humidity'],  # Information to display on hover
    title='City Map with Humidity as Point Size'
)

map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
ideal_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
ideal_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop results with null values
ideal_df = city_data_df.dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,11.76,71,100,7.76,SH,1724616370
1,1,diffa,13.3154,12.6113,26.18,74,100,5.47,NE,1724616372
2,2,toliara,-23.3500,43.6667,23.46,62,0,2.54,MG,1724616373
3,3,amasra,41.7463,32.3863,22.11,78,2,2.28,TR,1724616374
4,4,saldanha,-33.0117,17.9442,10.03,88,10,5.08,ZA,1724616031


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,71,
1,diffa,NE,13.3154,12.6113,74,
2,toliara,MG,-23.3500,43.6667,62,
3,amasra,TR,41.7463,32.3863,78,
4,saldanha,ZA,-33.0117,17.9442,88,
...,...,...,...,...,...,...
558,necochea,AR,-38.5473,-58.7368,64,
559,ixtapa,MX,20.7000,-105.2000,66,
560,kurchatov,RU,51.6601,35.6521,41,
561,san juanito,MX,28.0000,-107.5833,27,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
        'categories': 'accommodation.hotel',
        'apiKey': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
edinburgh of the seven seas - nearest hotel: No hotel found
diffa - nearest hotel: UNHCR guest house
toliara - nearest hotel: Ambary
amasra - nearest hotel: Liman Hotel
saldanha - nearest hotel: Hoedjesbaai
flying fish cove - nearest hotel: Christmas Island Lodge
saint-philippe - nearest hotel: Le Baril
biguacu - nearest hotel: Hotel Amorim
seven pagodas - nearest hotel: Radisson Temple Bay
papatowai - nearest hotel: No hotel found
kish - nearest hotel: هتل فارابی
grytviken - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
usogorsk - nearest hotel: Единственная
kamloops - nearest hotel: No hotel found
afaahiti - nearest hotel: Omati Lodge
ribeira grande - nearest hotel: Hospedaria JSF
sitka - nearest hotel: Aspen Suites Hotel
nemuro - nearest hotel: お宿エクハシ
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
lagoa da prata - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
tolanaro -

,City,Country,Lat,Lng,Humidity,Hotel Name
0,edinburgh of the seven seas,SH,-37.0676,-12.3116,71,No hotel found
1,diffa,NE,13.3154,12.6113,74,UNHCR guest house
2,toliara,MG,-23.3500,43.6667,62,Ambary
3,amasra,TR,41.7463,32.3863,78,Liman Hotel
4,saldanha,ZA,-33.0117,17.9442,88,Hoedjesbaai
...,...,...,...,...,...,...
558,necochea,AR,-38.5473,-58.7368,64,Cabañas Río Quequén
559,ixtapa,MX,20.7000,-105.2000,66,Ma. Cristina
560,kurchatov,RU,51.6601,35.6521,41,Россия
561,san juanito,MX,28.0000,-107.5833,27,Cabañas Presa Situriachi


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot_2 = city_data_df.hvplot.points(
    'Lng', 
    'Lat', 
    geo=True,
    tiles = "OSM",
    size='Humidity',  # Size of the points based on humidity
    color='City',     # Optionally color by city for better distinction
    hover_cols=['Hotel Name', 'Country'],  # Information to display on hover
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)